# Piotroski's F-SCORE Calculation

## Import Libraries

In [1]:
# import necessary libraries
import pandas as pd

## Read Dataset to Pandas Dataframe

In [2]:
# read fundamentals.csv file into a pandas dataframe
funda_df = pd.read_csv('fundamentals.csv')
# display funda_df
display(funda_df.head())

,CIK,Ticker_Symbol,Company_Name,Sector,Data_Date,Market_Capitalization,Total_Assets,Total_Liabilities,Net_Income_Before_Extra_Items,Cash_Flow_From_Operations,Total_Long_Term_Debt,Current_Assets,Current_Liabilities,Common_Shares_Outstanding,Total_Sales,Cost_Of_Goods_Sold
0,1750,AIR,AAR CORP,Capital Goods,2009-05-31,571.5948,1377.511,720.616,80.600,64.451,392.984,851.312,254.418,38.884,1423.976,1110.677
1,1750,AIR,AAR CORP,Capital Goods,2010-05-31,777.8348,1501.042,754.692,44.628,153.156,336.191,863.429,325.550,39.484,1352.151,1065.902
2,1750,AIR,AAR CORP,Capital Goods,2011-05-31,1049.8206,1703.727,868.438,70.860,108.598,329.802,913.985,416.010,39.781,1775.782,1408.071
3,1750,AIR,AAR CORP,Capital Goods,2012-05-31,485.2897,2195.653,1329.631,65.178,94.217,669.489,1063.272,473.226,40.273,2074.498,1662.408
4,1750,AIR,AAR CORP,Capital Goods,2013-05-31,790.0029,2136.900,1217.400,53.000,162.900,622.200,1033.700,389.000,39.382,2167.100,1714.500


In [3]:
# note: the cik of each company is usuallly 10 digits in length
# note: the leading 0's from some ciks have been removed
# note: therefore, these leading 0's have to be added again to ensure all ciks are of length 10

# add leading 0's to ciks that have a length less than 10
funda_df['CIK'] = funda_df['CIK'].apply(lambda x: '{0:0>10}'.format(x))

## Book-to-Market Ratio Quintiles

In [4]:
# note: piotroski calculates the book-to-market (b/m) ratio for each company and classifies them into quintiles
# note: therefore, the b/m ratio for each company in funda_df has to be calculated and classified
# note: b/m ratio = common shareholders' equity / market capitalization
# note: where common shareholders' equity = total assets - total liabilities

# calculate the b/m ratio for each company and store in a new column in funda_df 
funda_df['BM_Ratio'] = (funda_df['Total_Assets'] - funda_df['Total_Liabilities']) / funda_df['Market_Capitalization'] 

# check for any negative b/m ratio values in funda_df
# note: negative b/m ratio companies will be removed (if any exist)
neg_bm_count = funda_df.loc[funda_df['BM_Ratio']<0, 'BM_Ratio'].count()
print('There are %d rows containing negative b/m ratio values' % (neg_bm_count))

There are 395 rows containing negative b/m ratio values


In [5]:
# use b/m ratio to define b/m quintiles and store in a new column in funda_df (with negative b/m values)
funda_df['BM_Quintile'] = pd.qcut(funda_df['BM_Ratio'], 5, 
                                  labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

In [6]:
# find min b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('minimum b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].min())
    print('\n')

minimum b/m ratio for  Very Low :  -721.1686000122752


minimum b/m ratio for  Low :  0.21997382462498738


minimum b/m ratio for  Medium :  0.37272062726524147


minimum b/m ratio for  High :  0.5345481992129281


minimum b/m ratio for  Very High :  0.79977920876972




In [7]:
# find max b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('maximum b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].max())
    print('\n')

maximum b/m ratio for  Very Low :  0.2198595448185914


maximum b/m ratio for  Low :  0.3726678180165201


maximum b/m ratio for  Medium :  0.534494436672182


maximum b/m ratio for  High :  0.7997536900355039


maximum b/m ratio for  Very High :  199.68087080789144




In [8]:
# find mean b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('mean b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].mean())
    print('\n')

mean b/m ratio for  Very Low :  -0.7403826296443942


mean b/m ratio for  Low :  0.2967235276426672


mean b/m ratio for  Medium :  0.44748367992662524


mean b/m ratio for  High :  0.653656474032792


mean b/m ratio for  Very High :  1.7486575214390578




In [9]:
# remove companies with negative b/m ratio values in funda_df
neg_bm_idx = funda_df[funda_df['BM_Ratio']<0].index
funda_df = funda_df.drop(neg_bm_idx, axis=0).reset_index(drop=True)

funda_df = funda_df.drop(columns=['BM_Quintile'])

# use b/m ratio to define b/m quintiles and store in a new column in funda_df
funda_df['BM_Quintile'] = pd.qcut(funda_df['BM_Ratio'], 5, 
                                  labels=['Very Low', 'Low', 'Medium', 'High', 'Very High'])

In [10]:
# find minimum b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('minimum b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].min())
    print('\n')

minimum b/m ratio for  Very Low :  0.00029416521734059657


minimum b/m ratio for  Low :  0.24550994587877328


minimum b/m ratio for  Medium :  0.38868837504461984


minimum b/m ratio for  High :  0.5519084119552949


minimum b/m ratio for  Very High :  0.8169716198913278




In [11]:
# find max b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('minimum b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].max())
    print('\n')

minimum b/m ratio for  Very Low :  0.2454730790404054


minimum b/m ratio for  Low :  0.3885203126627278


minimum b/m ratio for  Medium :  0.5513956256415565


minimum b/m ratio for  High :  0.8169240468003308


minimum b/m ratio for  Very High :  199.68087080789144




In [12]:
# find mean b/m ratio of every b/m quintile
bm_quintiles = ['Very Low', 'Low', 'Medium', 'High', 'Very High']
for cat in bm_quintiles:
    print('minimum b/m ratio for ', cat, ': ', funda_df.loc[funda_df['BM_Quintile']==cat, 'BM_Ratio'].mean())
    print('\n')

minimum b/m ratio for  Very Low :  0.14788678976041433


minimum b/m ratio for  Low :  0.3173075588046172


minimum b/m ratio for  Medium :  0.4634486726781248


minimum b/m ratio for  High :  0.6689738421219509


minimum b/m ratio for  Very High :  1.7868637967264795




## Company Size Terciles

In [13]:
# note: piotroski classifies the size of a company into terciles
# note: therefore, the companies contained in funda_df have to be classified in size terciles

# use market capitalization to define comapny size terciles and store in a new column in funda_df
funda_df['Size_Tercile'] = pd.qcut(funda_df['Market_Capitalization'], 3, 
                                   labels=['Small', 'Medium', 'Large'])

In [14]:
# find min market capitalization of every company size tercile
company_size_terciles = ['Small', 'Medium', 'Large']
for cat in company_size_terciles:
    print('minimum market capitalization for ', cat, ': ', funda_df.loc[funda_df['Size_Tercile']==cat, 'Market_Capitalization'].min())
    print('\n')

minimum market capitalization for  Small :  3.1909


minimum market capitalization for  Medium :  1411.254


minimum market capitalization for  Large :  5543.1552




In [15]:
# find max market capitalization of every company size tercile
company_size_terciles = ['Small', 'Medium', 'Large']
for cat in company_size_terciles:
    print('maximum market capitalization for ', cat, ': ', funda_df.loc[funda_df['Size_Tercile']==cat, 'Market_Capitalization'].max())
    print('\n')

maximum market capitalization for  Small :  1409.5261


maximum market capitalization for  Medium :  5542.9432


maximum market capitalization for  Large :  438702.0




In [16]:
# find mean market capitalization of every company size tercile
company_size_terciles = ['Small', 'Medium', 'Large']
for cat in company_size_terciles:
    print('mean market capitalization for ', cat, ': ', funda_df.loc[funda_df['Size_Tercile']==cat, 'Market_Capitalization'].mean())
    print('\n')

mean market capitalization for  Small :  648.5828984558369


mean market capitalization for  Medium :  2981.2649394688083


mean market capitalization for  Large :  29840.454309326746




## Remove Companies with Less Than 3 Years of Data

In [17]:
# note: in order to calculate piotroski's f-score there has to be more than 2 years of data for each company

# filter ticker symbols with more than 2 years of data
funda_df = funda_df.groupby('Ticker_Symbol').filter(lambda x: len(x)>2).reset_index(drop=True)
# row count of funda_df after filtering
print('funda_df row count:', funda_df.shape[0])

funda_df row count: 9513


## Additional Required Variables

In [18]:
# note: additional variables from previous years are required to be stored in one instance (row)
# note: e.g. a new column for total assets from the previous year grouped by ticker is needed 

# obtain total assets for the previous year for each ticker and store in a new column in funda_df
total_assets_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                           x['Total_Assets'].shift(1))
funda_df['Total_Assets_PY1'] = total_assets_py1.reset_index(level=0, drop=True)

# obtain total assets for pre-previous year for each ticker and store in a new column in funda_df
total_assets_py2 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                           x['Total_Assets'].shift(2))
funda_df['Total_Assets_PY2'] = total_assets_py2.reset_index(level=0, drop=True)

# obtain total long term debt for previous year for each ticker and store in a new column in funda_df
total_lt_debt_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                            x['Total_Long_Term_Debt'].shift(1))
funda_df['Total_Long_Term_Debt_PY1'] = total_lt_debt_py1.reset_index(level=0, drop=True)

# obtain current assets for previous year for each ticker and store in a new column in funda_df
current_assets_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                             x['Current_Assets'].shift(1))
funda_df['Current_Assets_PY1'] = current_assets_py1.reset_index(level=0, drop=True)

# obtain current liabilities for previous year for each ticker and store in a new column in funda_df
current_liabilities_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                                  x['Current_Liabilities'].shift(1))
funda_df['Current_Liabilities_PY1'] = current_liabilities_py1.reset_index(level=0, drop=True)

# obtain common shares outstanding for previous year for each ticker and store in a new column in funda_df
csho_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                   x['Common_Shares_Outstanding'].shift(1))
funda_df['Common_Shares_Outstanding_PY1'] = csho_py1.reset_index(level=0, drop=True)

# obtain total sales for previous year for each ticker and store in a new column in funda_df
total_sales_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                          x['Total_Sales'].shift(1))
funda_df['Total_Sales_PY1'] = total_sales_py1.reset_index(level=0, drop=True)

# obtain cost of goods sold for previous year for each ticker and store in a new column in funda_df
cogs_py1 = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                   x['Cost_Of_Goods_Sold'].shift(1))
funda_df['Cost_Of_Goods_Sold_PY1'] = cogs_py1.reset_index(level=0, drop=True)

## Calculation of Piotroski's Nine Fundamental Measures & Scoring Functions

In [19]:
# note: Piotroski's F-SCORE is based on the following nine measures....
# --------------------profitability--------------------
# - return on assets (ROA)
# - cash flow from operations (CFO)
# - change in return on assets (ΔROA)
# - accruals (ACCRUAL)
# --------leverage, liquidity & source of funds--------
# - change in leverage (ΔLEVER)
# - change in liquidity (ΔLIQUID)
# - issuance of new equity (EQ_OFFER)
# -----------------operating efficiency----------------
# - change in margin (ΔMARGIN)
# - change in turnover (ΔTURN)
# note: each measure above is scored using a scoring function
# note: the scoring function for each fundamental measure is discussed below
# note: t denotes the time period; in this case, t represents years

# calculate return on assets and store in a new column in funda_df
# note: (ROA)_t = (net income before extraordinary items)_t / (total assets)_t-1
roa = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                              x['Net_Income_Before_Extra_Items'] /
                                                              x['Total_Assets_PY1'])
funda_df['ROA'] = roa.reset_index(level=0, drop=True)

# calculate cash flow from operations and store in a new column in funda_df
# note: (CFO)_t = (cash flow from operations)_t / (total assets)_t-1
cfo = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                              x['Cash_Flow_From_Operations'] /
                                                              x['Total_Assets_PY1'])
funda_df['CFO'] = cfo.reset_index(level=0, drop=True)

# calculate change in return on assets and store in a new column in funda_df
# note: (ΔROA)_t = (ROA)_t - (ROA)_t-1
croa = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                               x['ROA'] - x['ROA'].shift(1))
funda_df['CROA'] = croa.reset_index(level=0, drop=True)

# calculate accruals and store in a new column in funda_df
# note: (ACCRUAL)_t = (ROA)_t - (CFO)_t
accrual = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                  x['ROA'] - x['CFO'])
funda_df['ACCRUAL'] = accrual.reset_index(level=0, drop=True)

# calculate change in leverage and store in a new column in funda_df
# note: (ΔLEVER)_t = (long term debt)_t / 0.5 * ((total assets)_t + (total assets)_t-1) -
#                    (long term debt)_t-1 / 0.5 * ((total assets)_t-1 + (total assets)_t-2)
clever = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                 (x['Total_Long_Term_Debt'] /
                                                                  (0.5 * 
                                                                   (x['Total_Assets'] + 
                                                                    x['Total_Assets_PY1'])))
                                                                 -
                                                                 (x['Total_Long_Term_Debt_PY1'] /
                                                                  (0.5 * 
                                                                   (x['Total_Assets_PY1'] + 
                                                                    x['Total_Assets_PY2']))))
funda_df['CLEVER'] = clever.reset_index(level=0, drop=True)

# calculate change in liquidity and store in a new column in funda_df
# note: (ΔLIQUID)_t = (current assets)_t / (current liabilities)_t -
#                     (current assets)_t-1 / (current liabilities)_t-1
cliquid = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                  (x['Current_Assets'] / 
                                                                   x['Current_Liabilities']) 
                                                                  -
                                                                  (x['Current_Assets_PY1'] / 
                                                                   x['Current_Liabilities_PY1']))
funda_df['CLIQUID'] = cliquid.reset_index(level=0, drop=True)

# calculate issuance of new equity and store in a new column in funda_df
# note: (EQ_OFFER)_t = (common shares outstanding)_t - (common shares outstanding)_t-1
eq_offer = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                   x['Common_Shares_Outstanding']
                                                                   - 
                                                                   x['Common_Shares_Outstanding_PY1'])
funda_df['EQ_OFFER'] = eq_offer.reset_index(level=0, drop=True)

# calculate change in margin and store in a new column in funda_df
# note: (ΔMARGIN)_t = ((sales)_t - (cogs)_t) / (sales)_t -
#                     ((sales)_t-1 - (cogs)_t-1) / (sales)_t-1
cmargin = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                  ((x['Total_Sales'] - 
                                                                    x['Cost_Of_Goods_Sold']) /
                                                                   x['Total_Sales'])
                                                                  -
                                                                  ((x['Total_Sales_PY1'] -
                                                                    x['Cost_Of_Goods_Sold_PY1']) /
                                                                   x['Total_Sales_PY1']))
funda_df['CMARGIN'] = cmargin.reset_index(level=0, drop=True)

# calculate change in turnover and store in a new column in funda_df
# note: (ΔTURN)_t = (sales)_t / (total assets)_t-1 - (sales)_t-1 / (total assets)_t-2
cturn = funda_df.groupby('Ticker_Symbol', as_index=False).apply(lambda x:
                                                                (x['Total_Sales'] / x['Total_Assets_PY1'])
                                                                -
                                                                (x['Total_Sales_PY1'] / x['Total_Assets_PY2']))
funda_df['CTURN'] = cturn.reset_index(level=0, drop=True)

In [20]:
# note: the first 2 rows of a company's data in funda_df will contain missing values
# note: this is due to some of fundamental measures above containing variables that date back 2 years
# note: an example of this is change in turnover; it contains total assets that have to date back 2 years
# note: these rows with missing values will have to be removed

# remove any rows in funda_df that contain missing values and reset index
funda_df = funda_df.dropna().reset_index(drop=True)

In [21]:
# define scoring functions for each fundamental measure

# create new columns to store results from scoring functions in funda_df
funda_df['F_ROA'] = 0
funda_df['F_CFO'] = 0
funda_df['F_CROA'] = 0
funda_df['F_ACCRUAL'] = 0
funda_df['F_CLEVER'] = 0
funda_df['F_CLIQUID'] = 0
funda_df['F_EQ_OFFER'] = 0
funda_df['F_CMARGIN'] = 0
funda_df['F_CTURN'] = 0
# create a new column to store composite F-SCORE in funda_df
funda_df['F_SCORE'] = 0

# loop through each row of funda_df
for idx, row in funda_df.iterrows():
    
    # define ROA scoring function and store in F_ROA column in funda_df
    # note: F_ROA is defined as the ROA scoring function
    # note: if (ROA)_t > 0 then (F_ROA)_t = 1
    #       0 otherwise
    if row['ROA'] > 0:
        funda_df.at[idx, 'F_ROA'] = 1
    else:
        funda_df.at[idx, 'F_ROA'] = 0
        
    # define CFO scoring function and store in F_CFO column in funda_df
    # note: F_CFO is defined as the scoring function
    # note: if (CFO)_t > 0 then (F_CFO)_t = 1
    #       0 otherwise
    if row['CFO'] > 0:
        funda_df.at[idx, 'F_CFO'] = 1
    else:
        funda_df.at[idx, 'F_CFO'] = 0
    
    # define CROA scoring function and store in F_CROA column in funda_df
    # note: F_CROA is defined as the scoring function
    # note: if (CROA)_t > 0 then (F_CROA)_t = 1
    #       0 otherwise
    if row['CROA'] > 0:
        funda_df.at[idx, 'F_CROA'] = 1
    else:
        funda_df.at[idx, 'F_CROA'] = 0
    
    # define ACCRUAL scoring function and store in F_ACCRUAL column in funda_df
    # note: F_ACCRUAL is defined as the scoring function
    # note: if (CFO)_t > (ROA)_t then (F_ACCRUAL)_t = 1
    #       0 otherwise
    if row['CFO'] > row['ROA']:
        funda_df.at[idx, 'F_ACCRUAL'] = 1
    else:
        funda_df.at[idx, 'F_ACCRUAL'] = 0
    
    # define CLEVER scoring function and store in F_CLEVER column in funda_df
    # note: F_CLEVER is defined as the scoring function
    # note: if (CLEVER)_t < 0 then (F_CLEVER)_t = 1
    #       0 otherwise
    if row['CLEVER'] < 0:
        funda_df.at[idx, 'F_CLEVER'] = 1
    else:
        funda_df.at[idx, 'F_CLEVER'] = 0
    
    # define CLIQUID scoring function and store in F_CLIQUID column in funda_df
    # note: F_CLIQUID is defined as the scoring function
    # note: if (CLIQUID)_t > 0 then (F_CLIQUID)_t = 1
    #       0 otherwise
    if row['CLIQUID'] > 0:
        funda_df.at[idx, 'F_CLIQUID'] = 1
    else:
        funda_df.at[idx, 'F_CLIQUID'] = 0
    
    # define EQ_OFFER scoring function and store in F_EQ_OFFER column in funda_df
    # note: F_EQ_OFFER is defined as the scoring function
    # note: if (EQ_OFFER)_t <= 0 then (F_EQ_OFFER)_t = 1
    #       0 otherwise
    if row['EQ_OFFER'] <= 0:
        funda_df.at[idx, 'F_EQ_OFFER'] = 1
    else:
        funda_df.at[idx, 'F_EQ_OFFER'] = 0
    
    # define CMARGIN scoring function and store in F_CMARGIN column in funda_df
    # note: F_CMARGIN is defined as the scoring function
    # note: if (CMARGIN)_t > 0 then (F_CMARGIN)_t = 1
    #       0 otherwise
    if row['CMARGIN'] > 0:
        funda_df.at[idx, 'F_CMARGIN'] = 1
    else:
        funda_df.at[idx, 'F_CMARGIN'] = 0
    
    # define CTURN scoring function and store in F_CTURN column in funda_df
    # note: F_CTURN is defined as the scoring function
    # note: if (CTURN)_t > 0 then (F_CTURN) = 1
    #       0 otherwise
    if row['CTURN'] > 0:
        funda_df.at[idx, 'F_CTURN'] = 1
    else:
        funda_df.at[idx, 'F_CTURN'] = 0

# calculate composite F-SCORE
# note: F-SCORE = F_ROA + F_CFO + F_CROA + F_ACCRUAL + F_CLEVER + F_CLIQUID + F_EQ_OFFER + F_CMARGIN + F_CTURN
funda_df['F_SCORE'] = (funda_df['F_ROA'] + funda_df['F_CFO'] + funda_df['F_CROA'] + 
                       funda_df['F_ACCRUAL'] + funda_df['F_CLEVER'] + funda_df['F_CLIQUID'] + 
                       funda_df['F_EQ_OFFER'] + funda_df['F_CMARGIN'] + funda_df['F_CTURN'])

In [22]:
# rearrange column order of funda_df
funda_cols = ['CIK', 'Ticker_Symbol', 'Company_Name', 'Sector', 'Data_Date',
              'Market_Capitalization', 'Total_Assets', 'Total_Assets_PY1',
              'Total_Assets_PY2', 'Total_Liabilities',
              'Net_Income_Before_Extra_Items', 'Cash_Flow_From_Operations', 
              'Total_Long_Term_Debt', 'Total_Long_Term_Debt_PY1', 'Current_Assets', 
              'Current_Assets_PY1', 'Current_Liabilities', 'Current_Liabilities_PY1',
              'Common_Shares_Outstanding', 'Common_Shares_Outstanding_PY1',
              'Total_Sales', 'Total_Sales_PY1', 'Cost_Of_Goods_Sold', 
              'Cost_Of_Goods_Sold_PY1', 'Size_Tercile', 'BM_Ratio', 'BM_Quintile', 
              'ROA', 'F_ROA', 'CFO', 'F_CFO', 'CROA', 'F_CROA', 
              'ACCRUAL', 'F_ACCRUAL', 'CLEVER', 'F_CLEVER', 'CLIQUID', 'F_CLIQUID', 
              'EQ_OFFER', 'F_EQ_OFFER', 'CMARGIN', 'F_CMARGIN', 'CTURN', 'F_CTURN', 'F_SCORE']
funda_df = funda_df[funda_cols]

## Save Data to CSV File

In [23]:
# save funda_df to csv file
funda_df.to_csv('fundamentals_fscore.csv', index=False)